# Consignes

Intégrer le fichier USvideos.csv qui représente un ensemble de 8000 vidéos Youtube. 

Merger le fichier US_category_id.json pour récupérer le nom des catégories. Il conviendra de bien spécifier l'ID du document.


# Questions 
- 1) Récupérer toutes les vidéos de la chaîne Apple.
- 2) Compter le nombre de catégories différentes
- 3) Si vous ne l'avez pas déjà fait, découper les tags en listes et mettre à jour les tags de chacun des documents avec une requête update.
- 4) Récupérer les vidéos les plus vues.
- 5) Compter le nombre moyen de vues en fonction de la catégorie.
- 6) Récupérer les chaines Youtube avec la plus grande moyenne de likes.

In [1]:
import pandas as pd
import pymongo

In [2]:
client = pymongo.MongoClient("mongo")
database = client['exercices']
collection = database['youtube']

In [3]:
df_youtube = pd.read_csv("./data/USvideos.csv")
df_youtube.head(5)

,video_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...,4394029,320053,5931,46245,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,13.09
1,K4wEI5zhHB0,iPhone X — Introducing iPhone X — Apple,Apple,28,Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lig...,7860119,185853,26679,0,https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg,13.09
2,cLdxuaxaQwc,My Response,PewDiePie,22,[none],5845909,576597,39774,170708,https://i.ytimg.com/vi/cLdxuaxaQwc/default.jpg,13.09
3,WYYvHb03Eog,Apple iPhone X first look,The Verge,28,apple iphone x hands on|Apple iPhone X|iPhone ...,2642103,24975,4542,12829,https://i.ytimg.com/vi/WYYvHb03Eog/default.jpg,13.09
4,sjlHnJvXdQs,iPhone X (parody),jacksfilms,23,jacksfilms|parody|parodies|iphone|iphone x|iph...,1168130,96666,568,6666,https://i.ytimg.com/vi/sjlHnJvXdQs/default.jpg,13.09


In [4]:
c_tags=df_youtube["tags"]

for i in range (len(c_tags)):
    c_tags[i]=c_tags[i].split("|")
#c_tags.head()
df_youtube["tags"]=c_tags

<ipython-input-4-e8ffe8af026d>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  c_tags[i]=c_tags[i].split("|")


In [5]:
df_categories = pd.read_json("./data/US_category_id.json")
df_categories.head(5)

,kind,etag,items
0,youtube#videoCategoryListResponse,"""m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJv...","{'kind': 'youtube#videoCategory', 'etag': '""m2..."
1,youtube#videoCategoryListResponse,"""m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJv...","{'kind': 'youtube#videoCategory', 'etag': '""m2..."
2,youtube#videoCategoryListResponse,"""m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJv...","{'kind': 'youtube#videoCategory', 'etag': '""m2..."
3,youtube#videoCategoryListResponse,"""m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJv...","{'kind': 'youtube#videoCategory', 'etag': '""m2..."
4,youtube#videoCategoryListResponse,"""m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJv...","{'kind': 'youtube#videoCategory', 'etag': '""m2..."


## Question 0

### Netoyer les données

In [6]:
df_youtube.drop(['dislikes', 'comment_total', 'thumbnail_link', 'date'], axis='columns', inplace=True)
df_youtube.head(5)

,video_id,title,channel_title,category_id,tags,views,likes
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,"[logan paul vlog, logan paul, logan, paul, oly...",4394029,320053
1,K4wEI5zhHB0,iPhone X — Introducing iPhone X — Apple,Apple,28,"[Apple, iPhone 10, iPhone Ten, iPhone, Portrai...",7860119,185853
2,cLdxuaxaQwc,My Response,PewDiePie,22,[[none]],5845909,576597
3,WYYvHb03Eog,Apple iPhone X first look,The Verge,28,"[apple iphone x hands on, Apple iPhone X, iPho...",2642103,24975
4,sjlHnJvXdQs,iPhone X (parody),jacksfilms,23,"[jacksfilms, parody, parodies, iphone, iphone ...",1168130,96666


In [7]:
df_categ = pd.DataFrame(list(df_categories['items']))
df_categ.head(5)

,kind,etag,id,snippet
0,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/Xy1mB4_yLrHy_BmKm...",1,"{'channelId': 'UCBR8-60-B28hp2BmDPdntcQ', 'tit..."
1,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/UZ1oLIIz2dxIhO45Z...",2,"{'channelId': 'UCBR8-60-B28hp2BmDPdntcQ', 'tit..."
2,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/nqRIq97-xe5XRZTxb...",10,"{'channelId': 'UCBR8-60-B28hp2BmDPdntcQ', 'tit..."
3,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/HwXKamM1Q20q9BN-o...",15,"{'channelId': 'UCBR8-60-B28hp2BmDPdntcQ', 'tit..."
4,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/9GQMSRjrZdHeb1OEM...",17,"{'channelId': 'UCBR8-60-B28hp2BmDPdntcQ', 'tit..."


In [8]:
df_cat = pd.DataFrame(list(df_categ['snippet']))
df_cat.head(5)

,channelId,title,assignable
0,UCBR8-60-B28hp2BmDPdntcQ,Film & Animation,True
1,UCBR8-60-B28hp2BmDPdntcQ,Autos & Vehicles,True
2,UCBR8-60-B28hp2BmDPdntcQ,Music,True
3,UCBR8-60-B28hp2BmDPdntcQ,Pets & Animals,True
4,UCBR8-60-B28hp2BmDPdntcQ,Sports,True


In [9]:
df_categ.drop(['snippet'], axis='columns', inplace=True)
new_df_cat = pd.concat([df_categ, df_cat], axis=1)
new_df_cat = new_df_cat.rename(columns={"id":"category_id"})
new_df_cat["category_id"]=pd.to_numeric(new_df_cat["category_id"], downcast='integer')
new_df = pd.merge(df_youtube, new_df_cat, on='category_id')
new_df.head(5)

,video_id,title_x,channel_title,category_id,tags,views,likes,kind,etag,channelId,title_y,assignable
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,"[logan paul vlog, logan paul, logan, paul, oly...",4394029,320053,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/UVB9oxX2Bvqa_w_y3...",UCBR8-60-B28hp2BmDPdntcQ,Entertainment,True
1,4MkC65emkG4,Hand In Hand A Benefit For Hurricane Relief | MTV,MTV,24,"[mtv, video, online, official, tv, television,...",274358,9215,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/UVB9oxX2Bvqa_w_y3...",UCBR8-60-B28hp2BmDPdntcQ,Entertainment,True
2,vu_9muoxT50,Colin Cloud: Mind Reader Predicts Your Tweets ...,America's Got Talent,24,"[America's Got Talent 2017, america's got tale...",473691,14740,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/UVB9oxX2Bvqa_w_y3...",UCBR8-60-B28hp2BmDPdntcQ,Entertainment,True
3,ZQK1F0wz6z4,What Do You Want to Eat?!,Wong Fu Productions,24,"[panda, what should we eat, buzzfeed, comedy, ...",282858,14870,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/UVB9oxX2Bvqa_w_y3...",UCBR8-60-B28hp2BmDPdntcQ,Entertainment,True
4,5ywKal6-anc,Gigi Hadid Loses High Heel During Fashion Week...,TMZ,24,"[TMZ2016FS11221, TMZ, Hollywood, Celebrity, En...",703750,2921,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/UVB9oxX2Bvqa_w_y3...",UCBR8-60-B28hp2BmDPdntcQ,Entertainment,True


### Importer les données

In [12]:
collection.insert_many(new_df.to_dict(orient='records'))

In [11]:
#collection.delete_many({})

## Question 1  

In [13]:
list(collection.find({"channel_title":"Apple"}))

[{'_id': ObjectId('600384b540585c035b91ed3b'),
  'video_id': 'K4wEI5zhHB0',
  'title_x': 'iPhone X — Introducing iPhone X — Apple',
  'channel_title': 'Apple',
  'category_id': 28,
  'tags': ['Apple',
   'iPhone 10',
   'iPhone Ten',
   'iPhone',
   'Portrait Lighting',
   'A11 Bionic',
   'augmented reality',
   'emoji',
   'animoji',
   'Face ID',
   'Apple Pay',
   'camera',
   'smartphone'],
  'views': 7860119,
  'likes': 185853,
  'kind': 'youtube#videoCategory',
  'etag': '"m2yskBQFythfE4irbTIeOgYYfBU/w5HjcTD82G_XA3xBctS30zS-JpQ"',
  'channelId': 'UCBR8-60-B28hp2BmDPdntcQ',
  'title_y': 'Science & Technology',
  'assignable': True},
 {'_id': ObjectId('600384b540585c035b91ed4b'),
  'video_id': 'K4wEI5zhHB0',
  'title_x': 'iPhone X — Introducing iPhone X — Apple',
  'channel_title': 'Apple',
  'category_id': 28,
  'tags': ['Apple',
   'iPhone 10',
   'iPhone Ten',
   'iPhone',
   'Portrait Lighting',
   'A11 Bionic',
   'augmented reality',
   'emoji',
   'animoji',
   'Face ID',
 

## Question 2

In [14]:
categ=collection.distinct('title_y')
print(len(categ))

16


## Question 3

Question traitée dans le nettoyage de données

## Question 4

In [15]:
cur = collection.find().sort([("views", -1)])
list(cur)[0:15]

[{'_id': ObjectId('600384b540585c035b91fbbc'),
  'video_id': 'MBdVXkSdhwU',
  'title_x': "BTS (방탄소년단) 'DNA' Official MV",
  'channel_title': 'ibighit',
  'category_id': 10,
  'tags': ['BIGHIT', '빅히트', '방탄소년단', 'BTS', 'BANGTAN', '방탄'],
  'views': 41500672,
  'likes': 2010366,
  'kind': 'youtube#videoCategory',
  'etag': '"m2yskBQFythfE4irbTIeOgYYfBU/nqRIq97-xe5XRZTxbknKFVe5Lmg"',
  'channelId': 'UCBR8-60-B28hp2BmDPdntcQ',
  'title_y': 'Music',
  'assignable': True},
 {'_id': ObjectId('600384b540585c035b91fba8'),
  'video_id': 'MBdVXkSdhwU',
  'title_x': "BTS (방탄소년단) 'DNA' Official MV",
  'channel_title': 'ibighit',
  'category_id': 10,
  'tags': ['BIGHIT', '빅히트', '방탄소년단', 'BTS', 'BANGTAN', '방탄'],
  'views': 38013692,
  'likes': 1950475,
  'kind': 'youtube#videoCategory',
  'etag': '"m2yskBQFythfE4irbTIeOgYYfBU/nqRIq97-xe5XRZTxbknKFVe5Lmg"',
  'channelId': 'UCBR8-60-B28hp2BmDPdntcQ',
  'title_y': 'Music',
  'assignable': True},
 {'_id': ObjectId('600384b540585c035b91fa93'),
  'video_id':

## Question 5

In [16]:
cur = collection.aggregate([{"$group" : {"_id" : "$category_id", "nb_de_vues_moyen" : {"$avg" : "$views"}}}])
list(cur)

[{'_id': 23, 'nb_de_vues_moyen': 1240073.294039735},
 {'_id': 26, 'nb_de_vues_moyen': 537665.1806674339},
 {'_id': 15, 'nb_de_vues_moyen': 651404.3879310344},
 {'_id': 17, 'nb_de_vues_moyen': 728434.3243902439},
 {'_id': 10, 'nb_de_vues_moyen': 1176553.612},
 {'_id': 29, 'nb_de_vues_moyen': 1110334.2142857143},
 {'_id': 25, 'nb_de_vues_moyen': 540955.5569823436},
 {'_id': 43, 'nb_de_vues_moyen': 8492.5},
 {'_id': 27, 'nb_de_vues_moyen': 547582.4700598803},
 {'_id': 1, 'nb_de_vues_moyen': 1039472.6560846561},
 {'_id': 22, 'nb_de_vues_moyen': 971532.7573696146},
 {'_id': 2, 'nb_de_vues_moyen': 607693.8706896552},
 {'_id': 20, 'nb_de_vues_moyen': 681081.8170731707},
 {'_id': 24, 'nb_de_vues_moyen': 1154868.4397251718},
 {'_id': 19, 'nb_de_vues_moyen': 464041.0833333333},
 {'_id': 28, 'nb_de_vues_moyen': 924730.548828125}]

## Question 6 

In [17]:
cur = collection.aggregate([{"$group" : {"_id" : "$channel_title", "nb_de_likes_moyens" : {"$avg" : "$likes"}}},
                            {"$sort" : {"nb_de_likes_moyens" : -1}}])
list(cur)[0:15]

[{'_id': 'ZaynVEVO', 'nb_de_likes_moyens': 1431683.0},
 {'_id': 'ibighit', 'nb_de_likes_moyens': 1371766.25},
 {'_id': 'melanie martinez', 'nb_de_likes_moyens': 911871.5},
 {'_id': 'BETNetworks', 'nb_de_likes_moyens': 769687.625},
 {'_id': 'jypentertainment', 'nb_de_likes_moyens': 758826.6666666666},
 {'_id': 'Logan Paul Vlogs', 'nb_de_likes_moyens': 653443.5384615385},
 {'_id': 'Wengie', 'nb_de_likes_moyens': 567235.2},
 {'_id': 'Alesso', 'nb_de_likes_moyens': 509417.28571428574},
 {'_id': 'PewDiePie', 'nb_de_likes_moyens': 508647.8},
 {'_id': 'shakiraVEVO', 'nb_de_likes_moyens': 434302.14285714284},
 {'_id': 'Ed Sheeran', 'nb_de_likes_moyens': 395654.1666666667},
 {'_id': 'Dude Perfect', 'nb_de_likes_moyens': 389697.1111111111},
 {'_id': 'Conor Maynard', 'nb_de_likes_moyens': 389273.0},
 {'_id': 'TaylorSwiftVEVO', 'nb_de_likes_moyens': 367455.6666666667},
 {'_id': 'Liza Koshy', 'nb_de_likes_moyens': 365146.9583333333}]